In [117]:
import json
import boto3
import multiprocessing as mp
import numpy as np
import pickle

In [184]:
def matrix_read(filename):
    lines = open(filename, 'r').read().splitlines()
    A = []
    B = []
    matrix = A
    for line in lines:
        if line != "":            
            matrix.append(line)
        else:
            matrix = B
    A = '\n'.join(A)
    B = '\n'.join(B)
    return A, B


def matrix_dump(M):
    M_str = ""
    for line in M:
        M_str = M_str + ("\t".join(map(str, line.tolist()[0])) + "\n")
    return M_str
            

def matrix_parse(filename):
    lines = open(filename, 'r').read().splitlines()
    A = []
    B = []
    matrix = A
    for line in lines:
        if line != "":
            matrix.append(list(map(int, line.split("\t"))))
        else:
            matrix = B
    return A, B


def matrix_break_down(M):
    M_upper = np.vsplit(M ,2)[0]
    M_lower = np.vsplit(M, 2)[1]
    
    M11 = np.hsplit(M_upper, 2)[0]
    M12 = np.hsplit(M_upper, 2)[1]
    M21 = np.hsplit(M_lower, 2)[0]
    M22 = np.hsplit(M_lower, 2)[1]
    
    return M11, M12, M21, M22
    

def remote_matrix_op(A, B):
    A = matrix_dump(A)
    B = matrix_dump(B)
    payload = json.dumps({"A": A, "B": B})
    r = client.invoke(
        FunctionName='matrix_dot',
        Payload=payload)['Payload']
    return np.matrix(eval(json.loads(r.read().decode('utf8'))))

In [188]:
%%time

client = boto3.client('lambda')

A, B = matrix_parse('../data/250.in')
A = np.matrix(A)
B = np.matrix(B)

A11, A12, A21, A22 = matrix_break_down(A)
B11, B12, B21, B22 = matrix_break_down(B)

with mp.Pool(8) as p:
    r = p.starmap(remote_matrix_op, 
                    [(A11, B11), (A12, B21),
                     (A11, B12), (A12, B22),
                     (A21, B11), (A22, B21),
                     (A21, B12), (A22, B22)])

C0 = r[0] + r[1]
C1 = r[2] + r[3]  
C2 = r[4] + r[5]          
C3 = r[6] + r[7]

C_upper = np.concatenate((C0, C1), axis=1)
C_lower = np.concatenate((C2, C3), axis=1)
C = np.concatenate((C_upper, C_lower), axis=0)

CPU times: user 32 ms, sys: 96 ms, total: 128 ms
Wall time: 1.12 s


In [187]:
C

matrix([[1155471., 1307205.,  486386.,  604980.],
        [1083666.,  818081.,  554251.,  800302.],
        [1190768., 1400993.,  544053.,  690766.],
        [ 664342.,  890974.,  228973.,  334915.]])